In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse(r'C:\Users\cid2r\Downloads\db\discogs_20220201_masters.xml')
#tree = ET.parse('rel.xml')
root = tree.getroot()

In [3]:
for child in root:
    print(child.tag, child.attrib)

master {'id': '113'}
master {'id': '114'}
master {'id': '115'}
master {'id': '116'}
master {'id': '117'}
master {'id': '118'}
master {'id': '119'}
master {'id': '120'}
master {'id': '121'}
master {'id': '122'}
master {'id': '123'}
master {'id': '124'}
master {'id': '125'}
master {'id': '126'}
master {'id': '127'}
master {'id': '128'}
master {'id': '129'}
master {'id': '130'}
master {'id': '131'}
master {'id': '132'}
master {'id': '133'}
master {'id': '135'}
master {'id': '136'}
master {'id': '137'}
master {'id': '138'}
master {'id': '139'}
master {'id': '140'}
master {'id': '141'}
master {'id': '142'}
master {'id': '143'}
master {'id': '144'}
master {'id': '145'}
master {'id': '146'}
master {'id': '147'}
master {'id': '148'}
master {'id': '149'}
master {'id': '150'}
master {'id': '151'}
master {'id': '152'}
master {'id': '154'}
master {'id': '155'}
master {'id': '156'}
master {'id': '157'}
master {'id': '158'}
master {'id': '160'}
master {'id': '162'}
master {'id': '163'}
master {'id':

KeyboardInterrupt: 

In [4]:

import pandas as pd

import sys
if sys.version_info[0] == 3:
    from urllib.request import urlopen
else:
    from urllib import urlopen


In [ ]:
pip install python3-discogs-client

In [4]:
rows = []

for node in root:
    n_title = node.attrib["id"]
    #n_genre = node.find("genre").text
    rows.append({"title":n_title})
    

KeyboardInterrupt: 

In [5]:
import discogs_client

d = discogs_client.Client('ExampleApplication/0.1', user_token = 'WkcTWMWNFmwUFYHrtQIreHRnLmyFZTNnkflNHsgc')


In [10]:
d.master(121).stats

AttributeError: 'Master' object has no attribute 'stats'

In [7]:
master = []
masters = []


for release in root.iter('master'):
    id = release.attrib.get('id')
    if(int(id) > 74999):
        print(id)
    
        gens=[]
        for gen in release.iter('genre'):
           gens.append(gen.text)
    
        styls=[]
        for sty in release.iter('style'):
            styls.append(sty.text)
    
        master=[]
        imglnk = d.master(id).images
        if(imglnk != None):
            if('uri150' in imglnk[0]):
                uri = imglnk[0]['uri150']
    
                master = [id, gens, styls, uri]
    
                masters.append(master)

    
    if(int(id) > 84999):
        break

print("done! done!")

df = pd.DataFrame(masters, columns=['id', 'genre', 'styles', 'uri'])

from pandas import DataFrame

df.to_csv(r'C:\Users\cid2r\Documents\!VSCode\0220dbtest\data_75k_2_85k.csv')

75000
75001
75002
75003
75004
75005
75006
75007
75008
75009
75010
75011
75012
75013
75014
75015
75016
75017
75018
75020
75021
75022
75023
75024
75025
75026
75027
75028
75029
75030
75031
75032
75033
75034
75035
75036
75037
75038
75039
75040
75041
75042
75043
75044
75045
75046
75047
75048
75049
75050
75051
75052
75053
75054
75055
75056
75057
75058
75060
75061
75063
75064
75065
75066
75067
75069
75070
75071
75072
75073
75074
75075
75076
75077
75078
75079
75080
75081
75082
75083
75084
75085
75086
75087
75088
75089
75090
75091
75092
75093
75094
75095
75096
75097
75098
75099
75100
75101
75102
75103
75104
75105
75106
75107
75108
75109
75110
75111
75112
75113
75114
75115
75116
75117
75118
75119
75120
75121
75122
75123
75124
75125
75126
75127
75128
75129
75130
75131
75132
75133
75134
75135
75136
75137
75138
75139
75140
75141
75142
75143
75144
75145
75146
75147
75148
75149
75150
75151
75152
75153
75154
75155
75156
75157
75158
75159
75160
75161
75162
75163
75164
75165
75166
75167
75168
75169
7517

In [20]:
from pandas import DataFrame

df.to_csv(r'C:\Users\cid2r\Documents\!VSCode\0220dbtest\df.csv')

In [41]:
DISCOGS_KEY = 'OKxXReJmUJvMJRRsaOqr'
DISCOGS_SECRET = 'RormADbxpiujDCXDiBDwFGUFxKrVjkTK'

In [73]:
import discogs_client

d = discogs_client.Client('ExampleApplication/0.1', user_token = 'WkcTWMWNFmwUFYHrtQIreHRnLmyFZTNnkflNHsgc')

num = 121

asd = d.master(num).images[0]

type(asd)
#print(result)

print(asd['uri'])


https://i.discogs.com/V7w_rfIEDOXNGCclcLctqnrS3W1NfA45cGYKGWNlaxg/rs:fit/g:sm/q:90/h:600/w:600/czM6Ly9kaXNjb2dz/LWltYWdlcy9SLTEw/NDAzOC0xMjI3NDQ1/MTc5LmpwZWc.jpeg


In [ ]:
from bs4 import BeautifulSoup
import re
import urllib.request
from urllib.request import urlopen
import pandas as pd

url = open(r"C:\Users\cid2r\Documents\!VSCode\0220dbtest\rel.xml")

soup = BeautifulSoup(url, "html.parser")


data = {'id' : [], 'genre' : []}


for release in soup.select('release'):
    data['id'].append(release['id'])
    for gen in release.select('genre'):
        data['genre'].append(gen)

asdf = pd.DataFrame(data)

print(data)

#print(asdf)


In [19]:


for child in root:
    if(child.find("master_id") != None):
        print("=" * 10)
        print(child.attrib["id"])
        for gens in child.iter("genres"):
            for gen in gens.findall("genre"):
                print(gen.text)
        for mr in child.iter("master_id"):
            print(mr.text)
        ##child.write("output.xml", encoding="utf-8", xml_declaration=True)
        

1
jazz
soul
100
3
rock
300
